In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

import duckdb as db

from src.utils import duckdb_read_uc

In [ ]:
# Constants
# UC tables
CATALOG_NAME="your_uc_catalog"
SCHEMA_NAME="your_schema"
TABLE_NAME="your_table"

# Project 
DOTENV_PATH="devops/.env"

In [3]:
# Variable resolution and loading
load_dotenv(dotenv_path=Path(DOTENV_PATH))
table_uc_name=f"{CATALOG_NAME}.{SCHEMA_NAME}.{TABLE_NAME}"

1. Read table without column pruning nor predicate pushdown

In [ ]:
my_db_rel_all=duckdb_read_uc(table_uc_name)

In [ ]:
some_agg_from_all = db.query(
  """
  SELECT some_groupby_column, SUM(some_column_to_agg) as some_alias
  FROM my_db_rel_all
  GROUP BY some_groupby_column
  """
)

In [ ]:
query_results=some_agg_from_all.execute()

2. Read table with column pruning

In [ ]:
my_db_rel_column_pruned=(
    duckdb_read_uc(
        table_uc_name,
        columns=["some_groupby_column", "some_column_to_agg"]
    )
)

In [ ]:
some_agg_from_column_pruned = db.query(
  """
  SELECT some_groupby_column, SUM(some_column_to_agg) as some_alias
  FROM my_db_rel_column_pruned
  GROUP BY some_groupby_column
  """
)

In [ ]:
query_results=some_agg_from_column_pruned.execute()

3. Read table with column pruning and predicate pushdown (assume table partitioned)

In [ ]:
my_db_rel_small=(
    duckdb_read_uc(
        table_uc_name,
        partitions=[("some_partition_col", "=", "some_partition_val")],
        columns=["some_groupby_column", "some_column_to_agg"]
    )
)

In [ ]:
some_agg_from_small = db.query(
  """
  SELECT some_groupby_column, SUM(some_column_to_agg) as some_alias
  FROM my_db_rel_small
  GROUP BY some_groupby_column
  """
)

In [ ]:
query_results=some_agg_from_column_pruned.execute()

Compare sizes:

In [ ]:
def get_db_rel_size_in_gb(db_rel):
    return db_rel.arrow().nbytes / (1024 ** 3)

print(f"Size all: {get_db_rel_size_in_gb(my_db_rel_all)}")
print(f"Size column pruning: {get_db_rel_size_in_gb(my_db_rel_column_pruned)}")
print(f"Size column pruning and predicate pushdown: {get_db_rel_size_in_gb(my_db_rel_small)}")

Size all: 2.122442901134491
Size column pruning: 0.10247126780450344
Size column pruning and predicate pushdown: 0.04640738479793072
